This file contains code which was used during the development and testing the images and model.

In [2]:
import os
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt

In [3]:
label_dict = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 
    9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 
    17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 
    25: 'Z', 26: '0', 27: '1', 28: '2', 29: '3', 30: '4', 31: '5', 32: '6', 
    33: '7', 34: '8', 35: '9'
}

In [ ]:
def preprocess_image(image, size=(640, 640)):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    resized_image = cv2.resize(gray_image, size)          
    normalized_image = resized_image / 255.0              
    return normalized_image

combined_dataset_dir = './test'

for image_file in os.listdir(combined_dataset_dir):
    image_path = os.path.join(combined_dataset_dir, image_file)
    
    image = cv2.imread(image_path)
    if image is not None:
        image = preprocess_image(image)        

        cv2.imwrite(image_path, (image * 255).astype(np.uint8))

In [ ]:
model = YOLO("./runs/detect/train10/weights/best.pt")

image_path = './test/test.png'
original_image = cv2.imread(image_path)
results = model.predict(source=image_path, save=False, save_txt=False)

for result in results:
    boxes = result.boxes.xyxy.tolist()
    classes = result.boxes.cls.tolist()
    confidences = result.boxes.conf.tolist()
    
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        original_image = cv2.rectangle(original_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box with thickness 2
        label = f"Conf: {confidences[i]:.2f} Class: {label_dict[int(classes[i])]}"
        original_image = cv2.putText(original_image, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        plt.axis('off')
        plt.imshow(original_image)

In [ ]:
import cv2
import matplotlib.pyplot as plt

def show_bounding_boxes(image_path, label_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return
    
    height, width, _ = image.shape
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            box_width = float(parts[3])
            box_height = float(parts[4])
            
            x1 = int((x_center - box_width / 2) * width)
            y1 = int((y_center - box_height / 2) * height)
            x2 = int((x_center + box_width / 2) * width)
            y2 = int((y_center + box_height / 2) * height)
            
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            label = f"Class: {class_id}"
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            plt.axis('off')
            plt.imshow(image)

image_path = 'raw_data/yolo_format/hand1_0_bot_seg_1_cropped.jpeg'
label_path = 'raw_data/yolo_format/hand1_0_bot_seg_1_cropped.txt'
show_bounding_boxes(image_path, label_path)
